In [1]:
# @title 連接google drive
from google.colab import drive
drive.mount('/content/drive')

!nvidia-smi
import os
os.chdir('/content/drive/My Drive')
os.getcwd()
os.chdir('/content/drive/My Drive/112 Sesmester/data2')
os.getcwd()

Mounted at /content/drive
/bin/bash: line 1: nvidia-smi: command not found


'/content/drive/My Drive/112 Sesmester/data2'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from tqdm import tqdm

In [3]:

df = pd.read_csv("IsGreenOrder.csv")
print(df)
print(df.info())
# df.describe(include="all")
print("object行:\n",df.select_dtypes(include="object").columns)
print("缺失值:\n",df.isnull().sum())
print("唯一值:\n",df[:].nunique())

       Unnamed: 0       ID Delivery_person_ID  Delivery_person_Age  \
0               0  0x4607     INDORES13DEL02                  37.0   
1               1  0xb379     BANGRES18DEL02                  34.0   
2               2  0x5d6d     BANGRES19DEL01                  23.0   
3               3  0x7a6a    COIMBRES13DEL02                  38.0   
4               4  0x70a2     CHENRES12DEL01                  32.0   
...           ...      ...                ...                  ...   
40304       40304  0x1178   RANCHIRES16DEL01                  35.0   
40305       40305  0x7c09      JAPRES04DEL01                  30.0   
40306       40306  0x4f8d     CHENRES08DEL03                  30.0   
40307       40307  0x5eee    COIMBRES11DEL01                  20.0   
40308       40308  0x5fb2   RANCHIRES09DEL02                  23.0   

       Delivery_person_Ratings  Restaurant_latitude  Restaurant_longitude  \
0                          4.9            22.745049             75.892471   
1    

In [4]:
# @title 填補缺失值
# Analyzing the dataset to suggest suitable methods for imputing missing values in each column
data=df
# Checking for missing values in each column
# Analyzing the dataset to suggest suitable methods for imputing missing values in each column

# Checking for missing values in each column
missing_values = data.isnull().sum()
missing_values = missing_values[missing_values > 0]

# Checking the data types of columns with missing values
data_types = data[missing_values.index].dtypes

# Preparing a summary of columns with missing values, their data types, and suggested imputation methods
imputation_methods = []

for column, missing_count in missing_values.iteritems():
    dtype = data_types[column]
    if dtype == 'object':
        # Suggesting mode imputation for categorical data
        method = 'Mode (for categorical data)'
    elif dtype in ['int64', 'float64']:
        # Suggesting mean or median imputation for numerical data, depending on the distribution
        if data[column].skew() < 1 and data[column].skew() > -1:
            method = 'Mean (for approximately normal distribution)'
        else:
            method = 'Median (for skewed distribution)'
    else:
        method = 'Special consideration required'

    imputation_methods.append((column, dtype, missing_count, method))

imputation_summary = pd.DataFrame(imputation_methods, columns=['Column', 'Data Type', 'Missing Values', 'Suggested Method'])

imputation_summary.sort_values(by='Missing Values', ascending=False)


<ipython-input-4-1896cabd09c3>:17: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, missing_count in missing_values.iteritems():


,Column,Data Type,Missing Values,Suggested Method
4,City,object,1059,Mode (for categorical data)
2,multiple_deliveries,float64,855,Mean (for approximately normal distribution)
1,Delivery_person_Ratings,float64,242,Median (for skewed distribution)
3,Festival,object,206,Mode (for categorical data)
0,Delivery_person_Age,float64,200,Mean (for approximately normal distribution)


In [5]:
# @title 填補缺失值
# Filling missing values for each column based on the suggested methods

# Defining functions for different imputation methods
def impute_mode(column):
    return column.fillna(column.mode()[0])

def impute_mean(column):
    return column.fillna(column.mean())

def impute_median(column):
    return column.fillna(column.median())

# Applying the imputation methods to each column
for column, dtype, _, method in imputation_methods:
    if method == 'Mode (for categorical data)':
        data[column] = impute_mode(data[column])
    elif method == 'Mean (for approximately normal distribution)':
        data[column] = impute_mean(data[column])
    elif method == 'Median (for skewed distribution)':
        data[column] = impute_median(data[column])

# Checking if there are any missing values left
remaining_missing_values = data.isnull().sum().sum()

remaining_missing_values, data.head()


(0,
    Unnamed: 0       ID Delivery_person_ID  Delivery_person_Age  \
 0           0  0x4607     INDORES13DEL02                  37.0   
 1           1  0xb379     BANGRES18DEL02                  34.0   
 2           2  0x5d6d     BANGRES19DEL01                  23.0   
 3           3  0x7a6a    COIMBRES13DEL02                  38.0   
 4           4  0x70a2     CHENRES12DEL01                  32.0   
 
    Delivery_person_Ratings  Restaurant_latitude  Restaurant_longitude  \
 0                      4.9            22.745049             75.892471   
 1                      4.5            12.913041             77.683237   
 2                      4.4            12.914264             77.678400   
 3                      4.7            11.003669             76.976494   
 4                      4.6            12.972793             80.249982   
 
    Delivery_location_latitude  Delivery_location_longitude  Order_Date  ...  \
 0                   22.765049                    75.912471  2022-

In [6]:
# columns_to_drop = ['Unnamed: 0','ID', 'Delivery_person_ID', 'Order_Date', 'Time_Orderd', 'Time_Order_picked',
#                    'Route', 'Time_taken(min)','City','multiple_deliveries','Type_of_vehicle','Delivery_person_Age','Delivery_person_Ratings']
columns_to_drop = ['Unnamed: 0','ID', 'Delivery_person_ID', 'Order_Date', 'Time_Orderd', 'Time_Order_picked',
                   'Route', 'Time_taken(min)','Type_of_vehicle']
data = data.drop(columns_to_drop, axis=1)
data.columns

Index(['Delivery_person_Age', 'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Weatherconditions',
       'Road_traffic_density', 'Vehicle_condition', 'Type_of_order',
       'multiple_deliveries', 'Festival', 'City', 'Distance_km',
       'Time_Orderd_in_minutes', 'location_cluster', 'date_category',
       'IsGreenOrder'],
      dtype='object')

In [26]:
import numpy as np
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import f1_score, confusion_matrix, recall_score, roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from tqdm import tqdm
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
# from lightgbm import LGBMRegressor
# !pip install catboost
# from catboost import CatBoostRegressor

def train_and_evaluate(X_train, y_train, X_test, y_test):
    models = [
        ('XGBoost', xgb.DMatrix(X_train, label=y_train), xgb.DMatrix(X_test, label=y_test), True),
        ('LightGBM', lgb.Dataset(X_train, label=y_train), lgb.Dataset(X_test, label=y_test), True),
        ('Logistic Regression', LogisticRegression(), None, True),
        ('Decision Tree', DecisionTreeClassifier(), None, False),
        ('Random Forest', RandomForestClassifier(), None, False),
        # ('SVM', SVC(probability=True), None, False),
        ('KNN', KNeighborsClassifier(n_neighbors=3), None, False),
        # ('Neural Network', MLPClassifier(hidden_layer_sizes=(100,)), None, True)

    ]
    best_model = None
    best_model_name = ""
    best_score = float('-inf')
    for name, model, dtest, adjust_threshold in tqdm(models, desc="Training and evaluating models"):
        print(f"\n模型: {name}")


        # 打印 y_train 和 y_test 的分布
        print("\n")
        unique_values, counts = np.unique(y_train, return_counts=True)
        for value, count in zip(unique_values, counts):
          print(f"y_train值 {value} 出現了 {count} 次")

        unique_values, counts = np.unique(y_test, return_counts=True)
        for value, count in zip(unique_values, counts):
          print(f"y_test值 {value} 出現了 {count} 次")
        # print(X_train.describe())


        # 对于 XGBoost 和 LightGBM
        if name in ['XGBoost', 'LightGBM']:
            params = {'max_depth': 3, 'eta': 0.3, 'objective': 'binary:logistic'} if name == 'XGBoost' else {'objective': 'binary'}
            num_round = 100
            bst = xgb.train(params, model, num_round) if name == 'XGBoost' else lgb.train(params, model, num_boost_round=num_round)
            # y_test_pred = bst.predict(dtest.data if name == 'LightGBM' else dtest)
            y_test_pred = bst.predict(dtest.data if name == 'LightGBM' else dtest)
            model = bst

        # if name == 'XGBoost':
        #   dtrain = xgb.DMatrix(X_train, label=y_train)
        #   dtest = xgb.DMatrix(X_test, label=y_test)
        #   params = {'max_depth': 3, 'eta': 0.3, 'objective': 'binary:logistic'}
        #   num_round = 100
        #   bst = xgb.train(params, dtrain, num_round)
        #   y_test_pred = bst.predict(dtest)
        #   model=bst
        # elif name == 'LightGBM':
        #   model = lgb.LGBMClassifier(max_depth=3, learning_rate=0.1, n_estimators=100)
        #   model.fit(X_train, y_train)
        #   y_test_pred = model.predict_proba(X_test)[:, 1]
        # 对于其他模型
        else:
            model.fit(X_train, y_train)
            # y_test_pred = model.predict(X_test)  # 直接使用二元分类结果
            # y_test_pred = model.predict_proba(X_test)[:,1] if adjust_threshold else model.predict(X_test)
            y_test_pred = model.predict_proba(X_test)[:,1]

        # 创建一个 DataFrame 来比较预测概率和实际标签
        comparison_df = pd.DataFrame({
            'Predicted Probability': y_test_pred,
            'Actual Label': y_test
        })
        print(comparison_df.head())  # 打印前几行进行比较
        # # 评估性能
        if adjust_threshold:
            f1, conf_matrix, sensitivity, g_mean, roc_auc = evaluate_model_with_threshold(y_test, y_test_pred)
        else:
            f1, conf_matrix, sensitivity, g_mean, roc_auc = evaluate_model_with_threshold(y_test, y_test_pred)

        print(f"F1 分数: {f1}, 混淆矩阵: {conf_matrix}, Sensitivity: {sensitivity}, G-mean: {g_mean}, ROC-AUC: {roc_auc}")
        tn, fp, fn, tp = conf_matrix.ravel()
        print(f"  混淆矩阵: \n    TN: {tn}, FP: {fp}\n    FN: {fn}, TP: {tp}")
        # 检查是否是目前最好的模型
        if best_score < roc_auc:
            best_score = roc_auc
            best_model = model

            best_model_name = name

            print(f"best auc occur: {best_score}")
    print(best_model_name)
    return best_model_name,best_model,best_score

def evaluate_model_with_threshold(y_test, y_pred):
    # 确定最佳阈值
    best_threshold = 0.5
    best_f1 = 0
    for threshold in np.linspace(0, 1, 100):
        y_pred_binary = np.where(y_pred > threshold, 1, 0)
        f1 = f1_score(y_test, y_pred_binary)
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold

    y_pred_binary = np.where(y_pred > best_threshold, 1, 0)
    return calculate_metrics(y_test, y_pred_binary)

# def evaluate_model_without_threshold(y_test, y_pred):
#     return calculate_metrics(y_test, y_pred)

def calculate_metrics(y_test, y_pred_binary):
    f1 = f1_score(y_test, y_pred_binary)
    conf_matrix = confusion_matrix(y_test, y_pred_binary)
    sensitivity = recall_score(y_test, y_pred_binary)
    tn, fp, fn, tp = conf_matrix.ravel()
    specificity = tn / (tn + fp)
    g_mean = np.sqrt(sensitivity * specificity)
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_binary)
    roc_auc = auc(fpr, tpr)

    return f1, conf_matrix, sensitivity, g_mean, roc_auc



# 数据预处理
X = data.drop('IsGreenOrder', axis=1)
y = data['IsGreenOrder']

X_encoded = pd.get_dummies(X)
y_encoded = LabelEncoder().fit_transform(y)

# 分割数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.3, random_state=42)
# len(X_train), len(X_test), len(y_train), len(y_test)

# 定义不同的数据扩充方法
methods = [
    ('原始数据', X_train, y_train),
    ('随机过采样', *RandomOverSampler(random_state=42).fit_resample(X_train, y_train)),
    ('随机欠采样', *RandomUnderSampler(random_state=42).fit_resample(X_train, y_train)),
    ('SMOTE', *SMOTE(random_state=42).fit_resample(X_train, y_train)),
    ('ADASYN', *ADASYN(random_state=42).fit_resample(X_train, y_train))

]


# 存储结果的字典
results = {}
# 初始化最佳模型和性能指标
best_modelF = None
best_model_nameF = ""
best_scoreF = float('-inf')

best_model_method_name=""
# 进行模型训练和评估的循环
for method_name, X_augmented, y_augmented in tqdm(methods, desc="Training Models"):
    print(method_name)
    best_model_name,best_model, best_score = train_and_evaluate(X_augmented, y_augmented, X_test, y_test)
    if best_scoreF < best_score:
            best_scoreF = best_score
            best_modelF = best_model
            best_model_method_name= method_name
            best_model_nameF = best_model_name
            print(f"RR best AUC occur: {best_score}")
    print(type(best_modelF))

import joblib

if best_model_nameF in ['XGBoost', 'LightGBM']:
    # 使用模型特有的保存方法
    if best_model_nameF == 'XGBoost':
      best_modelF.save_model(f'{best_model_method_name}_{best_model_nameF}_best_model.json')
    if best_model_nameF == 'LightGBM':
      if isinstance(best_modelF, lgb.Booster):
          best_modelF.save_model(f'{best_model_method_name}_{best_model_nameF}_best_model.txt')
      elif isinstance(best_modelF, lgb.LGBMModel):
          best_modelF.booster_.save_model(f'{best_model_method_name}_{best_model_nameF}_best_model.txt')
      else:
          print("Error: best_modelF is not a LightGBM model.")
else:
    # 对于其他模型，使用 joblib
    joblib.dump(best_modelF, f'{best_model_method_name}_{best_model_nameF}_best_model.pkl')

print(f"最佳模型: {best_model_method_name} {best_model_nameF}，auc: {best_scoreF}")

Training Models:   0%|          | 0/5 [00:00<?, ?it/s]

原始数据



Training and evaluating models:   0%|          | 0/6 [00:00<?, ?it/s]


模型: XGBoost


y_train值 0 出現了 15758 次
y_train值 1 出現了 12458 次
y_test值 0 出現了 6806 次
y_test值 1 出現了 5287 次
   Predicted Probability  Actual Label
0               0.278839             0
1               0.113668             0
2               0.503029             0
3               0.251757             0
4               0.663200             0



Training and evaluating models:  17%|█▋        | 1/6 [00:00<00:04,  1.11it/s]

...


Training and evaluating models:  83%|████████▎ | 5/6 [00:12<00:03,  3.58s/it]

F1 分数: 0.6791272618958969, 混淆矩阵: [[3224 3582]
 [ 727 4560]], Sensitivity: 0.8624929071306979, G-mean: 0.6391890262762534, ROC-AUC: 0.6680962919432508
  混淆矩阵: 
    TN: 3224, FP: 3582
    FN: 727, TP: 4560

模型: KNN


y_train值 0 出現了 15758 次
y_train值 1 出現了 13056 次
y_test值 0 出現了 6806 次
y_test值 1 出現了 5287 次
   Predicted Probability  Actual Label
0               0.000000             0
1               0.000000             0
2               1.000000             0
3               0.000000             0
4               0.333333             0



Training Models: 100%|██████████| 5/5 [01:17<00:00, 15.53s/it]

F1 分数: 0.6228974498100922, 混淆矩阵: [[1941 4865]
 [ 695 4592]], Sensitivity: 0.8685454889351238, G-mean: 0.49769477320876443, ROC-AUC: 0.5768675137887491
  混淆矩阵: 
    TN: 1941, FP: 4865
    FN: 695, TP: 4592
LightGBM
<class 'lightgbm.basic.Booster'>


最佳模型: 原始数据 LightGBM，auc: 0.6913079203748891


In [21]:
print(type(best_modelF))

<class 'xgboost.core.DMatrix'>


In [ ]:
from datetime import datetime, timedelta

# Convert 'Order_Date' to datetime format
data['Order_Date'] = pd.to_datetime(data['Order_Date'])

# Sorting the data by 'Order_Date' and 'Time_Orderd_in_minutes'
sorted_data = data.sort_values(by=['Order_Date', 'Time_Orderd_in_minutes'])

# Calculate the time until next 'IsGreenOrder' == 1 in minutes
time_to_next_green_order = []
for index, row in sorted_data.iterrows():
    # Finding future orders where 'IsGreenOrder' == 1
    future_green_orders = sorted_data[(sorted_data['Order_Date'] > row['Order_Date']) |
                                      ((sorted_data['Order_Date'] == row['Order_Date']) &
                                       (sorted_data['Time_Orderd_in_minutes'] > row['Time_Orderd_in_minutes'])) &
                                      (sorted_data['IsGreenOrder'] == 1)]

    if not future_green_orders.empty:
        next_green_order = future_green_orders.iloc[0]
        # Calculate the time difference in minutes
        date_diff = (next_green_order['Order_Date'] - row['Order_Date']).days * 24 * 60  # Days to minutes
        time_diff = date_diff + (next_green_order['Time_Orderd_in_minutes'] - row['Time_Orderd_in_minutes'])
    else:
        time_diff = None  # No next green order found
    time_to_next_green_order.append(time_diff)
    print(f'append{time_diff}')
# Adding the calculated time to the sorted data
sorted_data['Time_to_Next_Green_Order'] = time_to_next_green_order

# Display the updated sorted dataframe
sorted_data[['Order_Date', 'Time_Orderd_in_minutes', 'IsGreenOrder', 'Time_to_Next_Green_Order']].head()


In [8]:
# @title 填補缺失值
# Analyzing the dataset to suggest suitable methods for imputing missing values in each column
data= pd.read_csv('/content/drive/My Drive/112 Sesmester/data2/featureS.csv')
columns_to_drop = ['Unnamed: 0', 'Time_Orderd_in_minutes','Time_taken(min)',
                   'Route', 'Time_taken(min)','City','multiple_deliveries','Type_of_vehicle']
data = data.drop(columns_to_drop, axis=1)
data.columns

data=df
# Checking for missing values in each column
# Analyzing the dataset to suggest suitable methods for imputing missing values in each column

# Checking for missing values in each column
missing_values = data.isnull().sum()
missing_values = missing_values[missing_values > 0]

# Checking the data types of columns with missing values
data_types = data[missing_values.index].dtypes

# Preparing a summary of columns with missing values, their data types, and suggested imputation methods
imputation_methods = []

for column, missing_count in missing_values.iteritems():
    dtype = data_types[column]
    if dtype == 'object':
        # Suggesting mode imputation for categorical data
        method = 'Mode (for categorical data)'
    elif dtype in ['int64', 'float64']:
        # Suggesting mean or median imputation for numerical data, depending on the distribution
        if data[column].skew() < 1 and data[column].skew() > -1:
            method = 'Mean (for approximately normal distribution)'
        else:
            method = 'Median (for skewed distribution)'
    else:
        method = 'Special consideration required'

    imputation_methods.append((column, dtype, missing_count, method))

imputation_summary = pd.DataFrame(imputation_methods, columns=['Column', 'Data Type', 'Missing Values', 'Suggested Method'])

imputation_summary.sort_values(by='Missing Values', ascending=False)
# @title 填補缺失值
# Filling missing values for each column based on the suggested methods

# Defining functions for different imputation methods
def impute_mode(column):
    return column.fillna(column.mode()[0])

def impute_mean(column):
    return column.fillna(column.mean())

def impute_median(column):
    return column.fillna(column.median())

# Applying the imputation methods to each column
for column, dtype, _, method in imputation_methods:
    if method == 'Mode (for categorical data)':
        data[column] = impute_mode(data[column])
    elif method == 'Mean (for approximately normal distribution)':
        data[column] = impute_mean(data[column])
    elif method == 'Median (for skewed distribution)':
        data[column] = impute_median(data[column])

# Checking if there are any missing values left
remaining_missing_values = data.isnull().sum().sum()

remaining_missing_values, data.head()


<ipython-input-8-a6bcb96b395a>:17: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, missing_count in missing_values.iteritems():


(0,
    Unnamed: 0       ID Delivery_person_ID  Delivery_person_Age  \
 0           0  0x4607     INDORES13DEL02                  37.0   
 1           1  0xb379     BANGRES18DEL02                  34.0   
 2           2  0x5d6d     BANGRES19DEL01                  23.0   
 3           3  0x7a6a    COIMBRES13DEL02                  38.0   
 4           4  0x70a2     CHENRES12DEL01                  32.0   
 
    Delivery_person_Ratings  Restaurant_latitude  Restaurant_longitude  \
 0                      4.9            22.745049             75.892471   
 1                      4.5            12.913041             77.683237   
 2                      4.4            12.914264             77.678400   
 3                      4.7            11.003669             76.976494   
 4                      4.6            12.972793             80.249982   
 
    Delivery_location_latitude  Delivery_location_longitude  Order_Date  ...  \
 0                   22.765049                    75.912471  2022-

In [ ]:
sorted_data['Time_to_Next_Green_Order'].value_counts()
sorted_data.to_csv('/content/drive/My Drive/112 Sesmester/data2/sorted_data')

In [ ]:
sorted_data[['Order_Date', 'Time_Orderd_in_minutes', 'IsGreenOrder', 'Time_to_Next_Green_Order']]


In [ ]:
# Sorting the data by 'Order_Date' and 'Time_Ordered'
sorted_data = data.sort_values(by=['Order_Date', 'Time_Ordered'])

# Display the first few rows of the sorted dataframe
sorted_data.head()
sorted_data[['Order_Date', 'Time_Ordered', 'IsGreenOrder', 'Time_to_Next_Green_Order']]

In [ ]:
data[ 'Time_to_Next_Green_Order'].value_counts()